In [1]:
import pandas as pd
import os
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

In [2]:
def fileaddres(path, spes):
    files = []
    #r = root, d=directory, f=files
    for r,d,f in os.walk(path):
        for file in f:
            if re.search(spes, file):
                files.append(os.path.join(r, file))
    return files

In [3]:
#our Current Directory of the files
files = []
path = "C:/Users/Redata/Downloads/FB/Churn Share" 
for r in os.listdir(path):
    if re.search(r"[0-9]", r):
        files.append(r)

In [4]:
#based on the order downloaded
def numoper(data):
    if "5" in data:
        return "Axis"
    elif "4" in data:
        return "XL"
    elif "3" in data:
        return "TRI"
    elif "2" in data:
        return "ISAT"
    elif "1" in data:
        return "TSEL"
    else:
        "Another"

In [5]:
data = {}
for i in files:
    satu = "C:/Users/Redata/Downloads/FB/Churn Share/" + i
    p = sorted(fileaddres(satu, r"csv"), reverse=True)
    satu = pd.read_csv(p[0])
    satu = satu.rename(columns={"value":"Smartfren"})
    for u in p[1:]:
        x = pd.read_csv(u)
        provider = numoper(u[-10:])
        x = x.rename(columns={"value":provider})
        satu = pd.merge(satu, x, left_on="name", right_on="name")
        satu[provider] = satu["Smartfren"] - satu[provider]
    f = pd.melt(satu, id_vars="name", value_vars=["Smartfren", "Axis", "XL", "TRI", "ISAT", "TSEL"], value_name=i)
    f = f.set_index(["name", "variable"])
    data[i] = f

In [6]:
x = data[list(data.keys())[0]]
for i in list(data.keys())[1:]:
    x = pd.concat([x, data[i]], axis=1)

x

Apr-19  Aug-19  Dec-19  Feb-19  Jan-19  Jan-20  Jul-19  \
name       variable                                                            
Bali       Axis         0.00    0.00    0.00    0.00    0.00    0.00    0.00   
           ISAT        17.13   16.50   17.31   21.18   19.33   16.58   15.60   
           Smartfren    6.33    8.77    8.87    4.34    4.83   10.17    8.15   
           TRI         10.79    9.79   10.47   10.70   11.56   10.00   10.87   
           TSEL        30.71   31.44   30.53   29.29   28.26   30.26   30.65   
...                      ...     ...     ...     ...     ...     ...     ...   
Yogyakarta ISAT        21.13   18.09   20.05   20.08   23.51   20.02   17.64   
           Smartfren   10.83   13.42   13.74    9.26    8.90   15.00   13.89   
           TRI         19.02   18.58   17.81   19.25   19.07   17.08   18.09   
           TSEL        25.89   27.44   27.38   26.07   23.42   27.61   27.17   
           XL          22.99   22.37   20.95   25.29   24.99   20.12   23.15   

                      Jun-19  Mar-19  May-19  Nov-19  Oct-19  Sep-19  
name       variable                                                   
Bali       Axis         0.00    0.00    0.00    0.00    0.00    0.00  
           ISAT        16.06   16.79   16.76   17.73   17.42   15.60  
           Smartfren    8.45    4.97    7.84    9.19   10.37    8.93  
           TRI         11.93   11.45   10.26    9.12    9.39   10.74  
           TSEL        30.00   31.52   30.45   30.87   29.73   32.06  
...                      ...     ...     ...     ...     ...     ...  
Yogyakarta ISAT        17.68   18.24   20.84   21.22   19.97   17.09  
           Smartfren   12.69   10.36   10.52   16.14   15.81   13.50  
           TRI         21.83   19.87   20.27   15.96   16.71   18.49  
           TSEL        25.04   27.41   24.84   28.05   26.09   29.91  
           XL          22.62   24.04   23.45   18.50   21.36   20.96  

[198 rows x 13 columns]

In [8]:
x.columns = pd.to_datetime([x.upper() for x in x.columns], format='%b-%y', yearfirst=False)
x = x.reindex(sorted(x.columns), axis=1)
x = x.reset_index()

#Exclude Axis provider
x = x[x["variable"] != "Axis"]
x

,name,variable,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00
1,Bali,ISAT,19.33,21.18,16.79,17.13,16.76,16.06,15.60,16.50,15.60,17.42,17.73,17.31,16.58
2,Bali,Smartfren,4.83,4.34,4.97,6.33,7.84,8.45,8.15,8.77,8.93,10.37,9.19,8.87,10.17
3,Bali,TRI,11.56,10.70,11.45,10.79,10.26,11.93,10.87,9.79,10.74,9.39,9.12,10.47,10.00
4,Bali,TSEL,28.26,29.29,31.52,30.71,30.45,30.00,30.65,31.44,32.06,29.73,30.87,30.53,30.26
5,Bali,XL,35.78,34.16,35.07,34.73,34.56,33.26,34.54,33.29,32.46,32.93,32.90,32.66,32.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Yogyakarta,ISAT,23.51,20.08,18.24,21.13,20.84,17.68,17.64,18.09,17.09,19.97,21.22,20.05,20.02
194,Yogyakarta,Smartfren,8.90,9.26,10.36,10.83,10.52,12.69,13.89,13.42,13.50,15.81,16.14,13.74,15.00
195,Yogyakarta,TRI,19.07,19.25,19.87,19.02,20.27,21.83,18.09,18.58,18.49,16.71,15.96,17.81,17.08
196,Yogyakarta,TSEL,23.42,26.07,27.41,25.89,24.84,25.04,27.17,27.44,29.91,26.09,28.05,27.38,27.61


In [9]:
#x.to_excel("Churn_share-in-ID-by-Id_province.xlsx")